# Book Recommendation System using HF

## 1) Load the dataset

In [2]:
from datasets import load_dataset
import torch.nn.functional as F
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel


In [27]:
books_data = load_dataset('vojtam/czech_books_descriptions', split="train+test")

In [28]:
books_data['text'][1:3]

['Volné pokračování historické detektivky Smrt nosí rudé škorně se opět odehrává ve středověké Praze, a to v prosinci roku 1399. Václav od Černého koně, podrychtář Starého Města pražského, řeší další případy. Nemá toho na svých bedrech málo: kromě zmizení biřice Vohnouta, požáru v radní síni, nepořádku v obecní pokladně a hledání viníků se musí potýkat s řadou dalších menších či větších problémů. Snaží se také odhalit identitu tajemných jurátů, kteří zřejmě stojí i za přepadením biskupova poselstva, a odkrýt podvody alchymisty s nekalou pověstí. A ke všemu občas na Václava svými tajnými ženskými zbraněmi skrytě zaútočí manželka Kateřina, toužící po nových šatech na očekávanou korunovaci...',
 'Román vypráví o inteligentním robotu NDR-113, který v budoucím 21. stoletím slouží svému pánu tak dobře, že dostane i lidské jméno Andrew. Díky výrobní závadě totiž Andrew vykročil do života vybaven lidskou schopností milovat a touhou po sebezdokonalování. Neřešitelné problémy však nastanou, když

## 2) Create embeddings

In [5]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Move model to GPU
model = model.to(device)

Using device: cuda


In [29]:
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def create_embeddings(tokenizer, model, input_texts, batch_size=32):
    embeddings_list = []
    
    for i in range(0, len(input_texts), batch_size):
        batch_texts = input_texts[i:i + batch_size]
        
        batch_dict = tokenizer(batch_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
        batch_dict = {k: v.to(device) for k, v in batch_dict.items()}
        
        # Get embeddings for batch
        with torch.no_grad(): 
            outputs = model(**batch_dict)
            batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
        
        embeddings_list.append(batch_embeddings.cpu())

        if (i + batch_size) % (batch_size * 10) == 0:
            print(f"Processed {i + batch_size}/{len(input_texts)} texts")
            
    return torch.cat(embeddings_list, dim=0)


In [30]:
# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ["passage: " + text for text in books_data['text']]

book_embeddings = create_embeddings(tokenizer, model, input_texts).cpu()
#scores = (embeddings[:2] @ embeddings[2:].T) * 100
#print(scores.tolist())
book_embeddings.shape

Processed 320/10091 texts
Processed 640/10091 texts
Processed 960/10091 texts
Processed 1280/10091 texts
Processed 1600/10091 texts
Processed 1920/10091 texts
Processed 2240/10091 texts
Processed 2560/10091 texts
Processed 2880/10091 texts
Processed 3200/10091 texts
Processed 3520/10091 texts
Processed 3840/10091 texts
Processed 4160/10091 texts
Processed 4480/10091 texts
Processed 4800/10091 texts
Processed 5120/10091 texts
Processed 5440/10091 texts
Processed 5760/10091 texts
Processed 6080/10091 texts
Processed 6400/10091 texts
Processed 6720/10091 texts
Processed 7040/10091 texts
Processed 7360/10091 texts
Processed 7680/10091 texts
Processed 8000/10091 texts
Processed 8320/10091 texts
Processed 8640/10091 texts
Processed 8960/10091 texts
Processed 9280/10091 texts
Processed 9600/10091 texts
Processed 9920/10091 texts


torch.Size([10091, 1024])

In [36]:
def find_similar(query: str, n = 5):
    input_query = "query: " + query
    query_embedding = create_embeddings(tokenizer, model, input_query)
    scores = ((query_embedding @ book_embeddings.T) * 100).detach().numpy()[0]
    top_indices = np.argsort(scores)[-n:][::-1]

    return top_indices

In [37]:
books_data.set_format('pandas')

In [39]:
indices = find_similar("drama z prostředí nemocnice", 10)
books_data[indices]

,title,author,text
0,Úsměvy smutných mužů,Josef Formánek,"Zápisky z léčebny.Když spadnete na dno, nezbýv..."
1,Připravte operační sál,Zdena Frýbová,Napínavý psychologický román s kriminální zápl...
2,"Bude to bolet, doktore?",Adam Kay,"Vítejte ve světě, kde je 97hodinový pracovní t..."
3,Tribunál smrti,Noah Gordon,"Není divu, že je lékařské prostředí čtenářsky ..."
4,Občas lžu,Alice Feeney,Amber je upoutána na nemocniční lůžko. Nemůže ...
5,Blízký konec,Robin Cook,Napínavý román z prostředí onkologického centr...
6,V bílém plášti,* antologie,Svorníkem povídkové sbírky jedněch z nejlepšíc...
7,Hlasy,Ursula Poznanski,"Lidé, kteří si divoce mumlají sami pro sebe.Kt..."
8,Všichni jsme utkáni z hvězd,Rowan Coleman,"Odejít z tohoto světa je snazší, když vás netr..."
9,Klíč,Kathryn Hughes,Zdravotní sestra Ellen Crosbyová nastoupí do n...


In [33]:
import pickle

with open('book_embeddings.pkl', 'wb') as file:
    pickle.dump(book_embeddings, file)

> This cell should contain anything needed to run gradio app (without needing to run any of the cells above)

In [9]:
import gradio as gr 
from datasets import load_dataset
import torch.nn.functional as F
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import pickle

with open('book_embeddings.pkl', 'rb') as file:
    book_embeddings = pickle.load(file)

model_checkpoint = 'intfloat/multilingual-e5-large'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

books_data = load_dataset('vojtam/czech_books_descriptions', split="train+test")
books_data.set_format('pandas')

def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def create_embeddings(tokenizer, model, input_texts, batch_size=32):
    embeddings_list = []
    
    for i in range(0, len(input_texts), batch_size):
        batch_texts = input_texts[i:i + batch_size]
        
        batch_dict = tokenizer(batch_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
        
        # Get embeddings for batch
        with torch.no_grad(): 
            outputs = model(**batch_dict)
            batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
        
        embeddings_list.append(batch_embeddings)

        if (i + batch_size) % (batch_size * 10) == 0:
            print(f"Processed {i + batch_size}/{len(input_texts)} texts")
            
    return torch.cat(embeddings_list, dim=0)

def find_similar_books(query: str, n = 5):
    input_query = "query: " + query
    query_embedding = create_embeddings(tokenizer, model, input_query)
    scores = ((query_embedding @ book_embeddings.T) * 100).detach().numpy()[0]
    top_indices = np.argsort(scores)[-n:][::-1]

    return  books_data[top_indices]



css = """
.full-height-gallery {
    height: calc(100vh - 250px);
    overflow-y: auto;
}
#submit-btn {
    background-color: #ff5b00;
    color: #ffffff;
}
"""

with gr.Blocks(css=css) as intf:
    with gr.Row():
        text_input = gr.Textbox(label="Popis knihy", info = "Zadejte popis knihy, kterou byste si chtěli přečíst a aplikace najde nejpodobněší knihy dle vašeho popisu", placeholder='Zadejte popis, například "drama z prostředí nemocnice"')
        n_books = gr.Number(value = 5, label = "Počet knih", info="Počet nejpodobnějších knih, které si přejete zobrazit", minimum = 1, step = 1)
    with gr.Row():
        submit_btn = gr.Button("Vyhledat knihy", elem_id="submit-btn")
        clear_btn = gr.Button("Smazat")
    with gr.Row():
        dataframe = gr.Dataframe(label="Podobné knihy", show_label=False, elem_classes = ["full-height-gallery"])
    
    submit_btn.click(fn=find_similar_books, inputs=[text_input, n_books], outputs=dataframe)
    clear_btn.click(fn=lambda: [None, []], inputs=None, outputs=[text_input, dataframe])

intf.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://4c5199aafb9c84e08b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
